In [46]:


import os

def list_all_folders(root_dir):
  for dirpath, dirnames, filenames in os.walk(root_dir):
    for dirname in dirnames:
      print(os.path.join(dirpath, dirname))

list_all_folders("D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos")


D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 1
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 10
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 11
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 12
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 13
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 14
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 15
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 16
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 17
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 18
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 19
D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos\Module 2
D:\Rec

In [47]:
# Rename all subfolders in each module to "Suspicious Behavior"
# I need to rename because the names of folders were not consistent across modules

import os

# Root directory
root_dir = r"D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos"
target_folder_name = "Suspicious Behavior"

# Loop through all immediate subfolders (modules)
for module in os.listdir(root_dir):
    module_path = os.path.join(root_dir, module)
    if os.path.isdir(module_path):
        for subfolder in os.listdir(module_path):
            subfolder_path = os.path.join(module_path, subfolder)
            # If subfolder is NOT already "Suspicious Behavior"
            if os.path.isdir(subfolder_path) and subfolder != target_folder_name:
                new_path = os.path.join(module_path, target_folder_name)
                # Rename it
                os.rename(subfolder_path, new_path)
                print(f"Renamed: {subfolder_path} -> {new_path}")

print("Done renaming subfolders to 'Suspicious Behavior'")


Done renaming subfolders to 'Suspicious Behavior'


In [48]:
# list all the annotated videos in the "Suspicious Behavior" folder and write them to a CSV file

import os
import csv

# Define root directory
root_dir = "D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos"
output_csv = "suspicious_videos - Ate Kath.csv"

# Define the name of the subfolder we're interested in
target_subfolder = "Suspicious Behavior"

# Allowed video extensions (you can customize this list)
video_extensions = ('.mp4', '.avi', '.mov', '.mkv')

# Collect video filenames
video_list = []

for module in os.listdir(root_dir):
    module_path = os.path.join(root_dir, module, target_subfolder)
    if os.path.isdir(module_path):
        for file in os.listdir(module_path):
            if file.lower().endswith(video_extensions):
                video_list.append([file])

# Write to CSV
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Video Name'])
    writer.writerows(video_list)

print(f"Done! {len(video_list)} video names written to '{output_csv}'")


Done! 629 video names written to 'suspicious_videos - Ate Kath.csv'


In [49]:
# List all the video names

import os
import csv

# Folder to search
root_dir = r"D:\Recordings\Annotations by domain\Kathlyn Ann Capalad\Segmented Videos"
output_csv = "video_names.csv"

# Video file extensions to include
video_extensions = ('.mp4', '.avi', '.mov', '.mkv')

# Collect video names only
video_names = []

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.lower().endswith(video_extensions):
            video_names.append([filename])

# Write to CSV
with open(output_csv, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Video Name'])
    writer.writerows(video_names)

print(f"Done! {len(video_names)} video name(s) saved to '{output_csv}'")


Done! 3470 video name(s) saved to 'video_names.csv'


In [50]:
# Convert the Excel file to CSV

import pandas as pd

# Load the Excel file (skip the first row with "Suspicious Behavior" etc.)
file_path = r"C:\Users\Raymond\Downloads\Modules - List - Susp.xlsx"
df = pd.read_excel(file_path, header=1)

# Flatten the entire DataFrame to a single list
video_names = df.values.flatten()

# Convert to Series, remove NaN, strip whitespace, and filter out empty strings
cleaned_videos = (
    pd.Series(video_names)
    .dropna()
    .astype(str)
    .str.strip()
)
cleaned_videos = cleaned_videos[cleaned_videos != ""]

# Add ".mp4" extension to each video name
cleaned_videos = cleaned_videos + ".mp4"

# Sort video names
sorted_videos = cleaned_videos.sort_values(ignore_index=True)

# Save to CSV
sorted_videos.to_frame(name="Video Name").to_csv("suspicious_videos - Maam Khaela.csv", index=False)

print("Maam Khaela's annotation: ", len(sorted_videos))


Maam Khaela's annotation:  461


In [51]:
import pandas as pd
import os

# List of annotator CSVs
annotator_csvs = [
    'suspicious_videos - Ate Kath.csv',
    'suspicious_videos - Maam Jas.csv',
    'suspicious_videos - Maam Khaela.csv'
]

# Load base video names
video_df = pd.read_csv('video_names.csv')

# Ensure column is named "Video Name" if needed
video_df.columns = ['Video Name']

# Merge each annotator's data
for csv_file in annotator_csvs:
    annotator_name = os.path.splitext(csv_file)[0]  # Get name without .csv
    annotator_df = pd.read_csv(csv_file)
    annotator_df.columns = ['Video Name']  # ensure column name matches

    # Add a column marking True if video is in this annotator's file
    video_df[annotator_name] = video_df['Video Name'].isin(annotator_df['Video Name'])

# Add agreement column: sum of True values across annotators
annotation_cols = annotator_csvs
annotation_cols = [os.path.splitext(f)[0] for f in annotation_cols]
video_df['Agreement'] = video_df[annotation_cols].sum(axis=1)

# Save to CSV
output_csv = "final_agreement_output.csv"
video_df.to_csv(output_csv, index=False)

print(output_csv)


final_agreement_output.csv


In [52]:
# # List the agreement between annotators
# # Videos marked as suspicious by each annotator

# import pandas as pd
# import os

# # List of annotator CSVs
# annotator_csvs = [
#     'suspicious_videos - Ate Kath.csv',
#     'suspicious_videos - Maam Jas.csv',
#     'suspicious_videos - Maam Khaela.csv'
# ]

# # Initialize an empty DataFrame to collect video names and annotations
# all_videos = pd.DataFrame()

# # Loop through each annotator CSV and collect data
# for csv_file in annotator_csvs:
#     annotator_name = os.path.splitext(csv_file)[0]  # Get name without .csv
#     annotator_df = pd.read_csv(csv_file)
#     annotator_df.columns = ['Video Name']  # Ensure column name is consistent

#     # Add a column for this annotator marking True if they flagged the video
#     annotator_df[annotator_name] = True

#     # Merge with the final DataFrame to accumulate all flagged videos
#     all_videos = pd.merge(all_videos, annotator_df[['Video Name', annotator_name]], on='Video Name', how='outer') if not all_videos.empty else annotator_df[['Video Name', annotator_name]]

# # Replace NaN values with False (if a video wasn't flagged by an annotator)
# all_videos = all_videos.fillna(False)

# # Add the "Agreement" column, summing the True values
# annotation_cols = [os.path.splitext(f)[0] for f in annotator_csvs]
# all_videos['Agreement'] = all_videos[annotation_cols].sum(axis=1)

# # Filter out videos with only 1 agreement
# all_videos_filtered = all_videos[all_videos['Agreement'] > 1]

# # Save to CSV
# output_csv = "filtered_agreement_output.csv"
# all_videos_filtered.to_csv(output_csv, index=False)

# print(f"Done! Filtered output saved to '{output_csv}'")


In [53]:
# # balance the dataset 
# # between videos marked as suspicious and those not marked by any annotator

# import pandas as pd

# # Load the CSV
# df = pd.read_csv("final_agreement_output.csv")

# # Identify rater columns
# rater_columns = df.columns[1:-1]

# # Convert string True/False to actual boolean
# df[rater_columns] = df[rater_columns].applymap(lambda x: True if str(x).lower() == 'true' else False)

# # Find rows where all are False
# all_false = df[df[rater_columns].sum(axis=1) == 0]

# # Find rows with at least one True
# has_true = df[df[rater_columns].sum(axis=1) > 0]

# # Balance the dataset
# min_count = min(len(all_false), len(has_true))

# balanced_df = pd.concat([
#     all_false.sample(n=min_count, random_state=42),
#     has_true.sample(n=min_count, random_state=42)
# ])

# # Shuffle the final balanced dataset (optional)
# balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Save or print
# print(balanced_df)

# # Optional: Save to a new CSV
# balanced_df.to_csv("balanced_agreement_output.csv", index=False)


In [54]:
# Calculate Fleiss' Kappa and Cohen's Kappa for inter-rater agreement

import pandas as pd
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score

# Load CSV
df = pd.read_csv("final_agreement_output.csv")

# Extract rater columns (assuming columns 1-3 are rater responses)
rater_columns = df.columns[1:-1]
ratings = df[rater_columns]

# Convert to boolean (True/False)
ratings = ratings.applymap(lambda x: True if str(x).lower() == 'true' else False)

# --- FLEISS' KAPPA ---
fleiss_matrix = ratings.apply(lambda row: pd.Series([sum(~row), sum(row)]), axis=1)
kappa_fleiss = fleiss_kappa(fleiss_matrix.values, method='fleiss')
print(f"Fleiss' Kappa: {kappa_fleiss:.4f}")

# --- COHEN'S KAPPA ---
rater1 = ratings.iloc[:, 0]
rater2 = ratings.iloc[:, 1]
rater3 = ratings.iloc[:, 2]

kappa_1_2 = cohen_kappa_score(rater1, rater2)
kappa_1_3 = cohen_kappa_score(rater1, rater3)
kappa_2_3 = cohen_kappa_score(rater2, rater3)

print(f"Cohen's Kappa (Rater 1 and 2): {kappa_1_2:.4f}")
print(f"Cohen's Kappa (Rater 1 and 3): {kappa_1_3:.4f}")
print(f"Cohen's Kappa (Rater 2 and 3): {kappa_2_3:.4f}")


C:\Users\Raymond\AppData\Local\Temp\ipykernel_20580\498009027.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ratings = ratings.applymap(lambda x: True if str(x).lower() == 'true' else False)


Fleiss' Kappa: 0.0703
Cohen's Kappa (Rater 1 and 2): 0.1147
Cohen's Kappa (Rater 1 and 3): 0.0778
Cohen's Kappa (Rater 2 and 3): 0.0741


In [ ]:
# # import pandas as pd

# # # Load the CSV
# # df = pd.read_csv('final_agreement_output.csv')

# # # Create subsets based on Agreement
# # df_0 = df[df['Agreement'] == 0][['Video Name']]
# # df_1 = df[df['Agreement'] == 1][['Video Name']]
# # df_2_3 = df[df['Agreement'].isin([2, 3])][['Video Name']]

# # # Save each to its own CSV
# # df_0.to_csv('agreement_0.csv', index=False)
# # df_1.to_csv('agreement_1.csv', index=False)
# # df_2_3.to_csv('agreement_2_3.csv', index=False)

# # print("Fucking done!")

# import pandas as pd

# # Load the CSV
# df = pd.read_csv('final_agreement_output.csv')

# # Create lists of video names for each agreement level
# videos_0 = sorted(df[df['Agreement'] == 0]['Video Name'].tolist())
# videos_1 = sorted(df[df['Agreement'] == 1]['Video Name'].tolist())
# videos_2_3 = sorted(df[df['Agreement'].isin([2, 3])]['Video Name'].tolist())

# print(len(videos_0), len(videos_1), len(videos_2_3))

# # Pad lists to the same length
# max_len = max(len(videos_0), len(videos_1), len(videos_2_3))
# videos_0 += [''] * (max_len - len(videos_0))
# videos_1 += [''] * (max_len - len(videos_1))
# videos_2_3 += [''] * (max_len - len(videos_2_3))

# # Create new DataFrame
# output_df = pd.DataFrame({
#     'Agreement_0': videos_0,
#     'Agreement_1': videos_1,
#     'Agreement_2_3': videos_2_3,
# })


# output_df.to_csv('grouped_agreements.csv', index=False)

# print("Fucking done!")


2478 825 167
Fucking done!


In [ ]:
import pandas as pd
import random
import re

# Load the CSV
df = pd.read_csv('final_agreement_output.csv')

# Get sorted video lists by agreement
videos_0 = sorted(df[df['Agreement'] == 0]['Video Name'].tolist())
videos_1 = sorted(df[df['Agreement'] == 1]['Video Name'].tolist())
videos_2_3 = sorted(df[df['Agreement'].isin([2, 3])]['Video Name'].tolist())

# Groupings of videos based on the same student
# I need to apply this groups because there were videos with different IDs but same student
# This is becuase there were time that the recording was interupted and the student had to continue the recording
# Resulting in different IDs but same student
group_definitions = {
    1: ["Video 01", "Video 02"],
    2: ["Video 04"],
    3: ["Video 05"],
    4: ["Video 06", "Video 10", "Video 11"],
    5: ["Video 07"],
    6: ["Video 09"],
    8: ["Video 12"],
    9: ["Video 13"],
    10: ["Video 14"],
    11: ["Video 15", "Video 16"],
    12: ["Video 20"],
    13: ["Video 21"],
    14: ["Video 22"]
}

# Reverse map: prefix → group number
video_to_group = {}
for group, prefixes in group_definitions.items():
    for prefix in prefixes:
        video_to_group[prefix] = group

# Helper: extract video prefix (e.g. "Video 01") from full name
def extract_prefix(video_name):
    match = re.match(r"(Video \d{2})", video_name)
    return match.group(1) if match else None

# Build group-wise dict for Agreement_0
grouped_0 = {}
for video in videos_0:
    prefix = extract_prefix(video)
    group = video_to_group.get(prefix)
    if group:
        grouped_0.setdefault(group, []).append(video)

# Match each video in Agreement_2_3 to same-group sample from Agreement_0
sampled = []
used_samples = set()
for v in videos_2_3:
    prefix = extract_prefix(v)
    group = video_to_group.get(prefix)
    candidates = grouped_0.get(group, [])
    # Pick a candidate that hasn't been used yet
    available = [c for c in candidates if c not in used_samples]
    if available:
        chosen = random.choice(available)
        sampled.append(chosen)
        used_samples.add(chosen)
    else:
        sampled.append("")  # No more videos available in that group

# Sort all lists
videos_0 = sorted(videos_0)
videos_1 = sorted(videos_1)
videos_2_3 = sorted(videos_2_3)
sampled = sorted(sampled)

# For reference only so I could track the numbers
print(len(videos_0), len(videos_1), len(videos_2_3), len(sampled))

# Pad all to same length
max_len = max(len(videos_0), len(videos_1), len(videos_2_3), len(sampled))
pad = lambda lst: lst + [''] * (max_len - len(lst))
videos_0 = pad(videos_0)
videos_1 = pad(videos_1)
videos_2_3 = pad(videos_2_3)
sampled = pad(sampled)

# Final DataFrame
output_df = pd.DataFrame({
    'Agreement_0': videos_0,
    'Agreement_1': videos_1,
    'Agreement_2_3': videos_2_3,
    'Sampled_from_Agreement_0': sampled
})

# output_df.to_csv('grouped_agreements.csv', index=False) 
# This is what I'm gonna use for training, create another csv so you won't ruin it


output_df.to_csv('your sample csv.csv', index=False)
print("Fucking done!")


2478 825 167 167
Fucking done!


In [1]:
# Move out the videos from subfolders to main folder

import os
import shutil

def move_videos_to_main_folder(base_folder):
    for subfolder in os.listdir(base_folder):
        subfolder_path = os.path.join(base_folder, subfolder)
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                input_path = os.path.join(subfolder_path, filename)
                if os.path.isfile(input_path) and filename.lower().endswith(".mp4"):
                    new_path = os.path.join(base_folder, filename)
                    shutil.move(input_path, new_path)
                    print(f"Moved: {filename} -> {base_folder}")

if __name__ == "__main__":
    base_folder = "D:\Recordings\Cleaned Videos\Segmented Videos - Copy"  # Base folder containing the videos
    move_videos_to_main_folder(base_folder)

Moved: Video 01-001.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-002.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-003.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-005.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-006.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-007.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-008.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-009.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-010.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-011.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-012.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-014.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy
Moved: Video 01-015.mp4 -> D:\Recordings

In [2]:
import pandas as pd
import os
import shutil

# PAths
csv_path = 'grouped_agreements.csv'
video_dir = r'D:\Recordings\Cleaned Videos\Segmented Videos - Copy'
suspicious_dir = r'D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious'
non_suspicious_dir = r'D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Non-suspicious'

df = pd.read_csv(csv_path)

# Create directories if they don't exist
os.makedirs(suspicious_dir, exist_ok=True)
os.makedirs(non_suspicious_dir, exist_ok=True)

# Function to move videos
def move_video(video_name, target_dir):
    src = os.path.join(video_dir, video_name)
    dst = os.path.join(target_dir, video_name)
    if os.path.exists(src):
        shutil.move(src, dst)
        print(f"Moved: {video_name} -> {target_dir}")
    else:
        print(f"Not found: {video_name}")

# Move videos to suspicious
for video in df['Agreement_2_3'].dropna():
    if video.strip():
        move_video(video.strip(), suspicious_dir)

# Move videos to non-suspicious
for video in df['Sampled_from_Agreement_0'].dropna():
    if video.strip():
        move_video(video.strip(), non_suspicious_dir)

print("Fucking done!")


Moved: Video 01-003.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-006.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-007.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-014.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-015.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-016.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-017.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-018.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-020.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-021.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious
Moved: Video 01-023.mp4 -> D:\Recordings\Cleaned Videos\Segmented Videos - Copy\Suspicious

In [13]:
# # Split the dataset into train, validation, and test sets
# This is the final step before training

import os
import shutil
import random
from math import floor

# Set your paths
source_dir = r"D:\Recordings\Cleaned Videos\Segmented Videos - Copy"
output_dir = "dataset"

# Set split ratios
train_ratio = 0.7
val_ratio = 0.20
test_ratio = 0.10

# Ensure reproducibility
random.seed(42)

# Class folders
classes = ["Suspicious", "Non-suspicious"]

for cls in classes:
    # Get all video files
    video_dir = os.path.join(source_dir, cls)
    files = [f for f in os.listdir(video_dir) if os.path.isfile(os.path.join(video_dir, f))]
    random.shuffle(files)

    # Calculate split sizes (using floor)
    total = len(files)
    train_count = floor(total * train_ratio)
    val_count = floor(total * val_ratio)
    test_count = total - train_count - val_count  # remainder goes to test

    # Split
    train_files = files[:train_count]
    val_files = files[train_count:train_count + val_count]
    test_files = files[train_count + val_count:]

    # Copy files
    for split, split_files in zip(["train", "val", "test"], [train_files, val_files, test_files]):
        split_dir = os.path.join(output_dir, split, cls)
        os.makedirs(split_dir, exist_ok=True)
        for f in split_files:
            src = os.path.join(video_dir, f)
            dst = os.path.join(split_dir, f)
            shutil.copy2(src, dst)

print("Done")


Done
